In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
df = pd.read_csv('diabetic_data.csv')
df = df.drop(['encounter_id', 'patient_nbr', 
            'weight', 'payer_code', 'medical_specialty', 
            'diag_1', 'diag_2', 'diag_3'], axis=1)
# df = df.loc[:, ~(df == '?').any()]
df_x = df.drop(['readmitted'], axis=1)
df_y = df['readmitted']
y_full = np.array(df_y.values)
y_full[y_full=='NO'] = 0
y_full[y_full=='>30'] = 1
y_full[y_full=='<30'] = 1
y_full = np.array(y_full).astype('int32')
categorical_variables = df_x.select_dtypes(include='object').columns.tolist()
continuous_variables = df_x.select_dtypes(exclude='object').columns.tolist()

ct_ss_ohe = ColumnTransformer(
    [('scaling', StandardScaler(), continuous_variables), 
    ('onehot', OneHotEncoder(sparse=False), categorical_variables)])
x_full = ct_ss_ohe.fit_transform(df_x)
x_train, x_test, y_train, y_test = train_test_split(x_full, y_full, stratify=y_full, random_state=0)

In [ ]:
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

(76324, 113)
(76324,)
(25442, 113)
(25442,)


In [ ]:
"""
    References:
    https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
    https://discuss.pytorch.org/t/whats-the-difference-between-nn-relu-vs-f-relu/27599/2
    https://youtu.be/oPhxf2fXHkQ
    https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
    https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
    https://pytorch.org/tutorials/beginner/basics/intro.html
    https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
    https://youtu.be/PXOzkkB5eH0
"""

class DiabetesDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x).float()
        self.y = torch.from_numpy(y).long()
        self.n_samples = x.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples
    


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer = nn.Linear(113, 25)
        self.output_layer = nn.Linear(25, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.hidden_layer(x)
        h_f = self.relu(x)
        x = self.output_layer(h_f)
        return x, h_f

class Model():
    def __init__(self, model, optimizer, loss_function):
        self.model = model
        self.optimizer = optimizer
        self.loss_function = loss_function
    
    def train(self, train_loader, epochs):
        self.train_loader = train_loader
        for epoch in range(epochs):
            for i, data in enumerate(train_loader):
                inputs = data[0].to(device)
                labels = data[1].to(device)
                self.optimizer.zero_grad()
                outputs, h_f = self.model(inputs)
                loss = self.loss_function(outputs, labels)
                loss.backward()
                self.optimizer.step()

                if (i+1)%100==0:
                    print (f'Epoch: {epoch+1}/{epochs}\t', end='', flush=True)
                    print (f'Batch: {i+1}/{len(train_loader)}\t', end='', flush=True)
                    print (f'Loss: {loss.item()}', flush=True)
            print ('=======================================================================================')
        return

    def test(self, test_loader):
        self.test_loader = test_loader
        probs = np.array([])
        with torch.no_grad():
            correct = 0
            total = 0
            for i, data in enumerate(test_loader):
                inputs = data[0].to(device)
                labels = data[1].to(device)
                outputs, h_f = self.model(inputs)
                if i==0:
                    probs = F.softmax(outputs, dim=1).numpy()
                else:
                    probs = np.vstack( (probs, F.softmax(outputs, dim=1).numpy() ))
                _, batch_pred = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (batch_pred==labels).sum().item()
        accuracy = (correct/total)*100
        return accuracy, probs
    
    def feature_extractor(self, data_loader):
        extracted_data = np.array([])
        with torch.no_grad():
            
            for i, data in enumerate(data_loader):
                inputs = data[0].to(device)
                labels = data[1].to(device)
                outputs, h_f = self.model(inputs)
                if i==0:
                    extracted_data = np.column_stack((h_f.numpy(), labels.numpy()))
                else:
                    extracted_data = np.vstack( (extracted_data, np.column_stack( ( h_f.numpy(), labels.numpy() ) ) ))
        return extracted_data

In [ ]:
batch_size = 256
lr = 0.001
epochs=100

In [ ]:
"""
    References:
    https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
    https://pytorch.org/docs/stable/data.html
"""
train_data = DiabetesDataset(x_train, y_train)
test_data = DiabetesDataset(x_test, y_test)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
net = Net().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()
model = Model(net, optimizer, loss_function)

In [ ]:
model.train(train_loader=train_loader, epochs=epochs)

Epoch: 1/100	Batch: 100/299	Loss: 0.6272361874580383
Epoch: 1/100	Batch: 200/299	Loss: 0.6332648992538452
Epoch: 2/100	Batch: 100/299	Loss: 0.6199131608009338
Epoch: 2/100	Batch: 200/299	Loss: 0.6527225375175476
Epoch: 3/100	Batch: 100/299	Loss: 0.6656703948974609
Epoch: 3/100	Batch: 200/299	Loss: 0.6196946501731873
Epoch: 4/100	Batch: 100/299	Loss: 0.6295291781425476
Epoch: 4/100	Batch: 200/299	Loss: 0.6226063370704651
Epoch: 5/100	Batch: 100/299	Loss: 0.6568787097930908
Epoch: 5/100	Batch: 200/299	Loss: 0.6696155667304993
Epoch: 6/100	Batch: 100/299	Loss: 0.6113044619560242
Epoch: 6/100	Batch: 200/299	Loss: 0.6545770764350891
Epoch: 7/100	Batch: 100/299	Loss: 0.6472380757331848
Epoch: 7/100	Batch: 200/299	Loss: 0.6289876103401184
Epoch: 8/100	Batch: 100/299	Loss: 0.6304786205291748
Epoch: 8/100	Batch: 200/299	Loss: 0.6574523448944092
Epoch: 9/100	Batch: 100/299	Loss: 0.6043490767478943
Epoch: 9/100	Batch: 200/299	Loss: 0.6112903952598572
Epoch: 10/100	Batch: 100/299	Loss: 0.624551951

In [ ]:
extracted_data = model.feature_extractor(train_loader)
print (extracted_data.shape)
print (extracted_data)
x_train = extracted_data[:, :-1]
y_train = extracted_data[:, -1]

(76324, 26)
[[2.26482391 0.         0.         ... 0.         0.05540963 0.        ]
 [0.17587906 0.         0.         ... 0.         0.         1.        ]
 [0.         0.52901733 0.3473174  ... 0.         0.         1.        ]
 ...
 [2.41943145 0.         0.         ... 0.         0.         0.        ]
 [1.86390448 0.         0.06162177 ... 0.89079309 0.         0.        ]
 [1.36132765 0.24491873 0.         ... 0.         0.         0.        ]]


In [ ]:
extracted_data = model.feature_extractor(test_loader)
print (extracted_data.shape)
print (extracted_data)
x_test = extracted_data[:, :-1]
y_test = extracted_data[:, -1]

(25442, 26)
[[1.75885141 0.         0.11609032 ... 0.         0.         0.        ]
 [1.93714237 0.42939928 0.40308356 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 ...
 [1.69368577 0.         0.38274473 ... 0.         0.         0.        ]
 [2.53201723 0.         0.         ... 0.09034063 0.49434343 1.        ]
 [1.82230234 0.61023349 0.         ... 0.         0.         0.        ]]


In [ ]:
from sklearn.svm import SVC
lsvm = SVC()
lsvm.C = 1
lsvm = lsvm.fit(x_train, y_train)
print (lsvm.score(x_train, y_train))
print (lsvm.score(x_test, y_test))

0.6517609139982181
0.6386683436836726
